# Часть 1 Бустинг (5 баллов)

В этой части будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [105]:
import pandas as pd
import numpy as np
import sklearn

df = pd.read_csv("ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла) Подготовка



*   Разделите выборку на train, val, test (80%, 10%, 10%)
*   Выдерите salary_in_usd в качестве таргета
*   Найдите и удалите признак, из-за которого возможен лик в данных


In [87]:
from sklearn.model_selection import train_test_split

y = df['salary_in_usd']
df.drop(['salary_in_usd', 'salary', 'salary_currency'], axis=1, inplace=True)
train, test, train_y, test_y = train_test_split(df, y, test_size=0.2, random_state=17)
test, val, test_y, val_y = train_test_split(test, test_y, test_size=0.5, random_state=17)
train.head(3)

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size
1995,2022,EN,FT,Data Engineer,US,0,US,M
3308,2022,SE,FT,Data Engineer,US,100,US,M
2965,2022,SE,FT,Machine Learning Infrastructure Engineer,US,100,US,M


In [88]:
print(f'exp: {df['experience_level'].unique()}')
print(f'size: {df['company_size'].unique()}')


exp: ['SE' 'MI' 'EN' 'EX']
size: ['L' 'S' 'M']


## Задание 2 (0.5 балла) Линейная модель


*   Закодируйте категориальные  признаки с помощью OneHotEncoder
*   Обучите модель линейной регрессии
*   Оцените  качество через MAPE и RMSE


In [99]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

def encode_train_test_val(train: pd.DataFrame,
                          test: pd.DataFrame,
                          val: pd.DataFrame,
                          encoder: OneHotEncoder | OrdinalEncoder,
                          column: str) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame):
    tmp_train, tmp_test, tmp_val = train.copy(), test.copy(), val.copy()

    train_res = encoder.fit_transform(train.loc[:, [column]])
    test_res = encoder.transform(test.loc[:, [column]])
    val_res = encoder.transform(val.loc[:, [column]])
    if isinstance(encoder, OneHotEncoder):
        new_columns = encoder.get_feature_names_out([column])

        train_res = pd.DataFrame(train_res, columns=new_columns, index=train.index)
        test_res = pd.DataFrame(test_res, columns=new_columns, index=test.index)
        val_res = pd.DataFrame(val_res, columns=new_columns, index=val.index)

        tmp_train.drop(columns=[column], inplace=True)
        tmp_test.drop(columns=[column], inplace=True)
        tmp_val.drop(columns=[column], inplace=True)

        tmp_train = pd.concat([tmp_train, train_res], axis=1)
        tmp_test = pd.concat([tmp_test, test_res], axis=1)
        tmp_val = pd.concat([tmp_val, val_res], axis=1)
    elif isinstance(encoder, OrdinalEncoder):
        tmp_train[column] = train_res
        tmp_test[column] = test_res
        tmp_val[column] = val_res

    return tmp_train, tmp_test, tmp_val

In [100]:
# Наши ординальные признаки
exp_categories = [['EN', 'MI', 'SE', 'EX']]
size_categories = [['S', 'M', 'L']]
ordinal = OrdinalEncoder(categories=exp_categories)
enc_train, enc_test, enc_val = encode_train_test_val(train, test, val, ordinal, column='experience_level')
ordinal = OrdinalEncoder(categories=size_categories)
enc_train, enc_test, enc_val = encode_train_test_val(enc_train, enc_test, enc_val, ordinal, column='company_size')

# Остальные
onehot = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
categorical_features = ['employment_type', 'job_title', 'employee_residence', 'company_location']
for categorical_feature in categorical_features:
    enc_train, enc_test, enc_val = encode_train_test_val(enc_train, enc_test, enc_val, onehot, categorical_feature)

enc_train.head(3)

,work_year,experience_level,remote_ratio,company_size,employment_type_CT,employment_type_FL,employment_type_FT,employment_type_PT,job_title_3D Computer Vision Researcher,job_title_AI Developer,...,company_location_RU,company_location_SE,company_location_SG,company_location_SI,company_location_SK,company_location_TH,company_location_TR,company_location_UA,company_location_US,company_location_VN
1995,2022,0.0,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3308,2022,2.0,100,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2965,2022,2.0,100,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [103]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, accuracy_score

linreg = LinearRegression()
linreg.fit(enc_train, train_y)
preds = linreg.predict(enc_test)

print('MAPE: ', mean_absolute_percentage_error(test_y, preds))
print('RMSE: ', mean_squared_error(test_y, preds))

MAPE:  0.3479265655786004
RMSE:  2612967589.7677026


### Изменения и их обоснование!!

experience_level намного логичнее кодировать ordinal_encoder, так как однозначно есть отношение порядка по уровню опыта

То же с company_size

## Задание 3 (0.5 балла) XGboost

Начнем с библиотеки xgboost.

Обучите модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [120]:
def build_res_df(max_depth: int,
                 learning_rate: float,
                 n_estimators: int,
                 gamma: float,
                 true_y: np.array,
                 predictions: np.array,
                 train_time: float,
                 prediction_time: float,
                 df: pd.DataFrame = None):
    if df is None:
        df = pd.DataFrame(columns = ['max_depth', 'learning_rate', 'n_estimators',
                                     'gamma',  'train_time', 'prediction_time', 'MAPE', 'RMSE',])
        df.set_index(['max_depth', 'learning_rate', 'n_estimators', 'gamma'], inplace=True)

    df.loc[max_depth, learning_rate, n_estimators, gamma] = [
        train_time,
        prediction_time,
        mean_absolute_percentage_error(true_y, predictions),
        mean_squared_error(true_y, predictions)
    ]

    return df

In [128]:
from xgboost.sklearn import XGBRegressor
from itertools import product
import time

params = {
    'max_depth' : [1, 3, 5, 7, 10, 20],
    'learning_rate' : [0.01, 0.05, 0.1, 0.5],
    'n_estimators' : [1, 10, 50, 100, 250],
    'gamma' : [0, 0.1, 0.5, 1]
}
def find_best_params_df(params_dict: dict)
    -> pd.DataFrame, float
    overall_start_time = time.time()
    df = None
    keys = list(params.keys())
    for max_depth, learning_rate, n_estimators, gamma in product(*[params[k] for k in keys]):
        reg = XGBRegressor(
            max_depth=max_depth,
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            gamma=gamma,
            random_state=17
        )
        start_train = time.time()
        reg.fit(enc_train, train_y)
        end_train = time.time()
    
        start_pred = time.time()
        preds = reg.predict(enc_val)
        end_pred = time.time()
    
        train_time = end_train - start_train
        predict_time = end_pred - start_pred
        df = build_res_df(max_depth,
                          learning_rate,
                          n_estimators,
                          gamma,
                          val_y,
                          preds,
                          train_time,
                          predict_time,
                          df)
    df.sort_values(by=['MAPE', 'RMSE'], ascending=True, inplace=True)
    overall_end_time = time.time()
    overall_time = overall_end_time - overall_start_time
    
    return df, overall_time
df, overall_time = find_best_params_df(params)
print(f'Всего гиперпараметры жбонкались: {overall_time} с')
print(f'За это время я выпил: {overall_time/60} смузи')

df.head(3)

Всего гиперпараметры жбонкались: 65.89727854728699 с
За это время я выпил: 1.0982879757881165 смузи


train_time  prediction_time  \
max_depth learning_rate n_estimators gamma                                
20        0.5           10           0.0      0.061159         0.006013   
                                     0.1      0.062616         0.006278   
                                     0.5      0.062048         0.005892   

                                                MAPE          RMSE  
max_depth learning_rate n_estimators gamma                          
20        0.5           10           0.0    0.327606  2.630021e+09  
                                     0.1    0.327606  2.630021e+09  
                                     0.5    0.327606  2.630021e+09

In [146]:
best_params = df.index[0]

reg = XGBRegressor(
    **dict(zip(df.index.names, df.index[0])),
    random_state=17
)
reg.fit(enc_train, train_y)
pred = reg.predict(enc_test)

print('MAPE: ', mean_absolute_percentage_error(test_y, pred))
print('RMSE: ', mean_squared_error(test_y, pred))

#TODO: вынести это в функцию, принимающую регрессор и параметры, чтобы дальше то же самое вызвать

MAPE:  0.3163871169090271
RMSE:  2345431808.0


## Задание 4 (1 балл) CatBoost

Теперь библиотека CatBoost.

Обучите модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [ ]:
from catboost import CatBoostRegressor

params = {
    'depth' : # -- YOUR CODE HERE -- ,
    'learning_rate' : # -- YOUR CODE HERE -- ,
    'iterations' : # -- YOUR CODE HERE -- ,
    # -- YOUR CODE HERE --
}

# -- YOUR CODE HERE --

In [ ]:
# -- YOUR CODE HERE --

print('MAPE: ', # -- YOUR CODE HERE -- )
print('RMSE: ', # -- YOUR CODE HERE -- )

Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучите catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features. Оцените качество и время. Стало ли лучше?

In [ ]:
from catboost import Pool

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 5 (0.5 балла) LightGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.


In [ ]:
from lightgbm import LGBMRegressor


params = {
    'max_depth' : # -- YOUR CODE HERE -- ,
    'learning_rate' : # -- YOUR CODE HERE -- ,
    'n_estimators' : # -- YOUR CODE HERE -- ,
    # -- YOUR CODE HERE --
}

# -- YOUR CODE HERE --

In [ ]:
# -- YOUR CODE HERE --

print('MAPE: ', # -- YOUR CODE HERE -- )
print('RMSE: ', # -- YOUR CODE HERE -- )

## Задание 6 (2 балла) Сравнение и выводы

Сравните модели бустинга и сделайте про них выводы, какая из моделей показала лучший/худший результат по качеству, скорости обучения и скорости предсказания? Как отличаются гиперпараметры для разных моделей?

**Ответ:** # -- YOUR ANSWER HERE --

# Часть 2 Кластеризация (5 баллов)

Будем работать с данными о том, каких исполнителей слушают пользователи музыкального сервиса.

Каждая строка таблицы - информация об одном пользователе. Каждый столбец - это исполнитель (The Beatles, Radiohead, etc.)

Для каждой пары (пользователь, исполнитель) в таблице стоит число - доля прослушивания этого исполнителя этим пользователем.


In [ ]:
import pandas as pd
ratings = pd.read_excel("https://github.com/evgpat/edu_stepik_rec_sys/blob/main/datasets/sample_matrix.xlsx?raw=true", engine='openpyxl')
ratings.head()

,user,the beatles,radiohead,deathcab for cutie,coldplay,modest mouse,sufjan stevens,dylan. bob,red hot clili peppers,pink fluid,...,municipal waste,townes van zandt,curtis mayfield,jewel,lamb,michal w. smith,群星,agalloch,meshuggah,yellowcard
0,0,NaN,0.020417,NaN,NaN,NaN,NaN,NaN,0.030496,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,0.184962,0.024561,NaN,NaN,0.136341,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,0.028635,NaN,NaN,NaN,0.024559,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.043529,0.086281,0.034590,0.016712,0.015935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Будем строить кластеризацию исполнителей: если двух исполнителей слушало много людей примерно одинаковую долю своего времени (то есть векторы близки в пространстве), то, возможно исполнители похожи. Эта информация может быть полезна при построении рекомендательных систем.

## Задание 1 (0.5 балла) Подготовка

Транспонируем матрицу ratings, чтобы по строкам стояли исполнители.

In [ ]:
# -- YOUR CODE HERE --

Выкиньте строку под названием `user`.

In [ ]:
# -- YOUR CODE HERE --

В таблице много пропусков, так как пользователи слушают не всех-всех исполнителей, чья музыка представлена в сервисе, а некоторое подмножество (обычно около 30 исполнителей)


Доля исполнителя в музыке, прослушанной  пользователем, равна 0, если пользователь никогда не слушал музыку данного музыканта, поэтому заполните пропуски нулями.



In [ ]:
# -- YOUR CODE HERE --
ratings.sample()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
ben harper,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Задание 2 (0.5 балла) Первая кластеризация

Примените KMeans с 5ю кластерами, сохраните полученные лейблы

In [ ]:
from sklearn.cluster import KMeans

# -- YOUR CODE HERE --

Выведите размеры кластеров. Полезной ли получилась кластеризация? Почему KMeans может выдать такой результат?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 3 (0.5 балла) Объяснение результатов

При кластеризации получилось $\geq 1$ кластера размера 1. Выведите исполнителей, которые составляют такие кластеры. Среди них должна быть группа The Beatles.

In [ ]:
# -- YOUR CODE HERE --

Изучите данные, почему именно The Beatles выделяется?

Подсказка: посмотрите на долю пользователей, которые слушают каждого исполнителя, среднюю долю прослушивания.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 4 (0.5 балла) Улучшение кластеризации

Попытаемся избавиться от этой проблемы: нормализуйте данные при помощи `normalize`.

In [ ]:
from sklearn.preprocessing import normalize

# -- YOUR CODE HERE --

Примените KMeans с 5ю кластерами на преобразованной матрице, посмотрите на их размеры. Стало ли лучше? Может ли кластеризация быть полезной теперь?

In [ ]:
# -- YOUR CODE HERE --

**Ответ** # -- YOUR ANSWER HERE --

## Задание 5 (1 балл) Центроиды

Выведите для каждого кластера названия топ-10 исполнителей, ближайших к центроиду по косинусной мере. Проинтерпретируйте результат. Что можно сказать о смысле кластеров?

In [ ]:
from scipy.spatial.distance import cosine


centroids = km.cluster_centers_

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 6 (1 балл) Визуализация

Хотелось бы как-то визуализировать полученную кластеризацию. Постройте точечные графики `plt.scatter` для нескольких пар признаков исполнителей, покрасив точки в цвета кластеров. Почему визуализации получились такими? Хорошо ли они отражают разделение на кластеры? Почему?

In [ ]:
import matplotlib.pyplot as plt

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Для визуализации данных высокой размерности существует метод t-SNE (стохастическое вложение соседей с t-распределением). Данный метод является нелинейным методом снижения размерности: каждый объект высокой размерности будет моделироваться объектов более низкой (например, 2) размерности таким образом, чтобы похожие объекты моделировались близкими, непохожие - далекими с большой вероятностью.

Примените `TSNE` из библиотеки `sklearn` и визуализируйте полученные объекты, покрасив их в цвета их кластеров

In [ ]:
from sklearn.manifold import TSNE

# -- YOUR CODE HERE --

## Задание 7 (1 балл) Подбор гиперпараметров

Подберите оптимальное количество кластеров (максимум 100 кластеров) с использованием индекса Силуэта. Зафиксируйте `random_state=42`

In [ ]:
from sklearn.metrics import silhouette_score

# -- YOUR CODE HERE --

Выведите исполнителей, ближайших с центроидам (аналогично заданию 5). Как соотносятся результаты? Остался ли смысл кластеров прежним? Расскажите про смысл 1-2 интересных кластеров, если он изменился и кластеров слишком много, чтобы рассказать про все.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Сделайте t-SNE визуализацию полученной кластеризации.

In [ ]:
# -- YOUR CODE HERE --

Если кластеров получилось слишком много и визуально цвета плохо отличаются, покрасьте только какой-нибудь интересный кластер из задания выше (`c = (labels == i)`). Хорошо ли этот кластер отражается в визуализации?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --